## NY State of Health Plan Scraper

This notebook scrapes health insurance plan data from the NY State of Health website.

### Strategy:
1. **Scrape plan listings** from `viewAllPlans` for each county
2. **Extract plan IDs** from the HTML
3. **Scrape individual plan details** from `getPlanDetails`
4. **Parse all data** into pandas DataFrames

### URL Structure:
- **All Plans**: `/individual/viewAllPlans?county={county}&coverageTier={tier}&planYearTxt={year}`
- **Plan Details**: `/individual/getPlanDetails/{planId}?county={county}&coverageTier={tier}&planYear={year}`


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock


def get_page_with_session(url, params=None, max_retries=3):
    """Fetch a page with session and cookie handling"""
    for attempt in range(max_retries):
        try:
            session = requests.Session()
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
            }
            
            # First visit homepage to get cookies
            session.get(base_url, headers=headers, timeout=30)
            time.sleep(1)
            
            # Now fetch the actual page
            response = session.get(url, params=params, headers=headers, timeout=30)
            
            if response.status_code == 200:
                return response
            else:
                print(f"  ⚠ Status {response.status_code} (attempt {attempt+1}/{max_retries})")
                time.sleep(2)
        except Exception as e:
            print(f"  ⚠ Error: {e} (attempt {attempt+1}/{max_retries})")
            time.sleep(2)
    
    return None

print("✓ Defined get_page_with_session()")


def extract_plan_ids_from_html(html_content):
    """Extract plan IDs from the viewAllPlans page HTML"""
    soup = BeautifulSoup(html_content, 'html.parser')
    plan_ids = []
    
    # Pattern 1: Card IDs like id="card_135116"
    for card in soup.find_all('div', id=re.compile(r'^card_\d+$')):
        match = re.search(r'card_(\d+)', card['id'])
        if match:
            plan_ids.append(match.group(1))
    
    # Pattern 2: onclick handlers with getPlanDetails
    # Example: onclick="getPlanDetails(&#39;getPlanDetails/&#39;+135116+&#39;?county=..."
    onclick_pattern = r"getPlanDetails\([^+]*\+(\d+)\+"
    for elem in soup.find_all(onclick=True):
        onclick_text = elem.get('onclick', '')
        matches = re.findall(onclick_pattern, onclick_text)
        plan_ids.extend(matches)
    
    # Pattern 3: Direct links with getPlanDetails in href
    for link in soup.find_all('a', href=True):
        if 'getPlanDetails' in link['href']:
            match = re.search(r'/getPlanDetails/(\d+)', link['href'])
            if match:
                plan_ids.append(match.group(1))
    
    # Pattern 4: Search in raw HTML for plan IDs (fallback)
    if not plan_ids:
        matches = re.findall(r'(?:planId|plan_id|card_)[\"\']?\s*[:\-]?\s*[\"\']?(\d{5,7})', html_content)
        plan_ids.extend(matches)
    
    # Remove duplicates while preserving order
    unique_ids = list(dict.fromkeys(plan_ids))
    return unique_ids

print("✓ Defined extract_plan_ids_from_html()")

def parse_plan_details(html_content, plan_id, county, coverage_tier, plan_year):
    """Parse plan details from getPlanDetails page HTML"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Initialize with basic info
    plan_data = {
        'plan_id': plan_id,
        'full_plan_id': None,
        'county': county,
        'coverage_tier': coverage_tier,
        'plan_year': plan_year,
        'plan_name': None,
        'premium': None,
        'metal_level': None,
        'allows_health_savings_account': None,
        'monthly_premium': None,
        'persons_covered': None,
        'individual_annual_deductible': None,
        'group_annual_deductible': None,
        'individual_out_of_pocket_maximum': None,
        'group_out_of_pocket_maximum': None,
    }
    
    # Extract all table data into a temporary dict (lowercase keys)
    table_data = {}
    tables = soup.find_all('table')
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all(['th', 'td'])
            if len(cells) >= 2:
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                if key and value:
                    # Store with normalized key
                    normalized_key = key.lower().replace('-', '_').replace(' ', '_').replace('(', '').replace(')', '')
                    table_data[normalized_key] = value
    
    # Helper function to clean dollar values
    def clean_dollar_value(val):
        if not val:
            return None
        val_clean = val.replace('$', '').replace(',', '').strip()
        if val_clean.lower() in ['not applicable', 'n/a', 'na', 'none', '']:
            return None
        try:
            return float(val_clean)
        except:
            return val_clean
    
    # Extract full_plan_id (like "56184NY0150015")
    for key in ['plan_id', 'planid', 'plan_identifier', 'hios_plan_id', 'planID']:
        if key in table_data:
            plan_data['full_plan_id'] = table_data[key]
            break
    
    # Extract plan_name
    for selector in ['h1', 'h2', '.plan-name', '.plan-title']:
        elem = soup.select_one(selector)
        if elem and elem.text.strip():
            plan_data['plan_name'] = elem.text.strip()
            break
    
    # Extract metal_level
    metal_elem = soup.find('span', class_=re.compile(r'metal'))
    if metal_elem:
        plan_data['metal_level'] = metal_elem.text.strip()
    else:
        for key in ['metal_level', 'metal_tier', 'plan_metal_level']:
            if key in table_data:
                plan_data['metal_level'] = table_data[key]
                break
    
    # Extract allows_health_savings_account
    for key in ['allows_health_savings_account', 'health_savings_account', 'hsa_eligible', 'hsa']:
        if key in table_data:
            val = table_data[key].lower()
            plan_data['allows_health_savings_account'] = 'yes' in val or 'eligible' in val
            break
    
    # Extract premium / monthly_premium
    for key in ['monthly_premium', 'premium', 'premium_per_month']:
        if key in table_data:
            plan_data['premium'] = clean_dollar_value(table_data[key])
            plan_data['monthly_premium'] = plan_data['premium']
            break
    
    # Extract persons_covered
    for key in ['persons_covered', 'coverage_tier', 'covered_persons']:
        if key in table_data:
            plan_data['persons_covered'] = table_data[key]
            break
    
    # Extract individual_annual_deductible
    for key in ['individual_annual_deductible', 'individual_deductible', 'deductible']:
        if key in table_data:
            plan_data['individual_annual_deductible'] = clean_dollar_value(table_data[key])
            break
    
    # Extract group_annual_deductible
    for key in ['group_annual_deductible', 'family_annual_deductible', 'family_deductible']:
        if key in table_data:
            plan_data['group_annual_deductible'] = clean_dollar_value(table_data[key])
            break
    
    # Extract individual_out_of_pocket_maximum
    for key in ['individual_outofpocket_maximum', 'individual_out_of_pocket_maximum', 
                'individual_oop_maximum', 'individual_maximum_outofpocket']:
        if key in table_data:
            plan_data['individual_out_of_pocket_maximum'] = clean_dollar_value(table_data[key])
            break
    
    # Extract group_out_of_pocket_maximum
    for key in ['group_outofpocket_maximum', 'group_out_of_pocket_maximum',
                'family_outofpocket_maximum', 'family_out_of_pocket_maximum',
                'family_oop_maximum', 'group_maximum_outofpocket']:
        if key in table_data:
            plan_data['group_out_of_pocket_maximum'] = clean_dollar_value(table_data[key])
            break
    
    return plan_data

print("✓ Defined parse_plan_details() - extracts 15 fields")


def scrape_plans_for_county(county, coverage_tier="INDIVIDUAL", plan_year=2026):
    """Scrape all plans using the correct getMorePlansData endpoint"""
    print(f"\n{'='*60}")
    print(f"Scraping: {county} | {coverage_tier} | {plan_year}")
    print(f"{'='*60}")
    
    # Setup URLs
    get_more_plans_url = f"{base_url}/individual/getMorePlansData"
    
    # Create session
    session = requests.Session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    }
    
    # Step 1: Get initial page (page 1 with first 6 plans)
    print(f"  Loading initial page...")
    initial_url = f"{view_plans_url}?county={county}&coverageTier={coverage_tier}&planYearTxt={plan_year}"
    response = session.get(initial_url, headers=headers, timeout=30)
    
    if not response or response.status_code != 200:
        print(f"  ✗ Failed to load initial page")
        return []
    
    # Get total count and first plans
    soup = BeautifulSoup(response.text, 'html.parser')
    total_elem = soup.find('input', {'id': 'totalPlanCount'})
    total_plans = int(total_elem.get('value', 0)) if total_elem else None
    
    all_plan_ids = extract_plan_ids_from_html(response.text)
    print(f"    ✓ Page 1: {len(all_plan_ids)} plans (Total: {total_plans})")
    
    # Step 2: Load remaining pages using getMorePlansData
    # pageNo starts at 1 (page 0 was the initial load)
    page_num = 1
    page_size = 6
    
    while True:
        # Check if we've collected all plans
        if total_plans and len(set(all_plan_ids)) >= total_plans:
            print(f"    ℹ Collected all {total_plans} plans")
            break
        
        # Fetch next page
        print(f"  Loading page {page_num}...", end='', flush=True)
        params = {
            'county': county,
            'coverageTier': coverage_tier,
            'planYearTxt': str(plan_year),
            'outOfNetwork': 'false',
            'pageNo': str(page_num),
            'pageSize': str(page_size)
        }
        
        try:
            response = session.get(get_more_plans_url, params=params, headers=headers, timeout=30)
            if response.status_code != 200:
                print(f" ✗ Status {response.status_code}")
                break
                
            page_plan_ids = extract_plan_ids_from_html(response.text)
            
            if not page_plan_ids:
                print(f" (no more plans)")
                break
            
            # Track unique plans
            before_count = len(set(all_plan_ids))
            all_plan_ids.extend(page_plan_ids)
            after_count = len(set(all_plan_ids))
            new_count = after_count - before_count
            
            print(f" ✓ {len(page_plan_ids)} plans ({new_count} new)")
            
            if new_count == 0:
                print(f"    ℹ No new plans, stopping")
                break
                
        except Exception as e:
            print(f" ✗ Error: {e}")
            break
        
        page_num += 1
        time.sleep(0.5)  # Rate limiting
        
        # Safety limit
        if page_num > 50:
            print(f"    ⚠ Safety limit reached")
            break
    
    # Remove duplicates
    unique_plan_ids = list(dict.fromkeys(all_plan_ids))
    print(f"\n✓ Found {len(unique_plan_ids)} unique plan IDs")
    
    # Step 3: Scrape each plan's details
    all_plans = []
    for idx, plan_id in enumerate(unique_plan_ids, 1):
        print(f"  [{idx}/{len(unique_plan_ids)}] Scraping plan {plan_id}...", end='', flush=True)
        
        plan_url = f"{plan_details_url}/{plan_id}"
        plan_params = {
            'county': county,
            'coverageTier': coverage_tier,
            'planYear': str(plan_year),
        }
        
        try:
            response = session.get(plan_url, params=plan_params, headers=headers, timeout=30)
            if response.status_code == 200:
                plan_data = parse_plan_details(response.text, plan_id, county, coverage_tier, plan_year)
                all_plans.append(plan_data)
                print(" ✓")
            else:
                print(f" ✗ ({response.status_code})")
        except Exception as e:
            print(f" ✗ ({str(e)[:20]})")
        
        # Rate limiting
        time.sleep(0.3)
    
    print(f"\n✓ Scraped {len(all_plans)} plans for {county}")
    return all_plans

print("✓ Defined scrape_plans_for_county()")


# Base URLs
base_url = "https://nystateofhealth.ny.gov"
view_plans_url = f"{base_url}/individual/viewAllPlans"
plan_details_url = f"{base_url}/individual/getPlanDetails"
# NY Counties list

NY_COUNTIES = [
    "Albany", "Allegany", "Bronx", "Broome", "Cattaraugus", "Cayuga", "Chautauqua",
    "Chemung", "Chenango", "Clinton", "Columbia", "Cortland", "Delaware", "Dutchess",
    "Erie", "Essex", "Franklin", "Fulton", "Genesee", "Greene", "Hamilton", "Herkimer",
    "Jefferson", "Kings", "Lewis", "Livingston", "Madison", "Monroe", "Montgomery",
    "Nassau", "New York", "Niagara", "Oneida", "Onondaga", "Ontario", "Orange",
    "Orleans", "Oswego", "Otsego", "Putnam", "Queens", "Rensselaer", "Richmond",
    "Rockland", "Saratoga", "Schenectady", "Schoharie", "Schuyler", "Seneca",
    "St.%20Lawrence", "Steuben", "Suffolk", "Sullivan", "Tioga", "Tompkins", "Ulster",
    "Warren", "Washington", "Wayne", "Westchester", "Wyoming", "Yates"
]

# Coverage tiers
COVERAGE_TIERS = ["INDIVIDUAL", "COUPLE", "FAMILY", "SINGLE_PARENT"]

# Plan years
PLAN_YEARS = [2026, 2025]

print(f"✓ Loaded {len(NY_COUNTIES)} NY counties")
print(f"✓ Coverage tiers: {COVERAGE_TIERS}")
print(f"✓ Plan years: {PLAN_YEARS}")



✓ Defined get_page_with_session()
✓ Defined extract_plan_ids_from_html()
✓ Defined parse_plan_details() - extracts 15 fields
✓ Defined scrape_plans_for_county()
✓ Loaded 62 NY counties
✓ Coverage tiers: ['INDIVIDUAL', 'COUPLE', 'FAMILY', 'SINGLE_PARENT']
✓ Plan years: [2026, 2025]


---
## Scraper Test

Use any County name from the counties list above, and any year from the years list above

In [2]:
# TEST: Scrape Albany to verify we get all 109 plans
print("🧪 TEST: Scraping Albany with the fixed scraper")
print("This should now get all 109 plans!\n")

test_plans = None # scrape_plans_for_county("St.%20Lawrence", 'INDIVIDUAL', 2026)

if test_plans:
    df_test = pd.DataFrame(test_plans)
    print(f"\n" + "="*60)
    print(f"✅ SUCCESS! Scraped {len(df_test)} plans")
    print(f"Expected: ~109 plans")
    print(f"\nDataFrame: {df_test.shape[0]} rows × {df_test.shape[1]} columns")
    print(f"\nRandom 5 plans:")
    display(df_test[['plan_id', 'metal_level', 'monthly_premium', 
                     'individual_annual_deductible', 'individual_out_of_pocket_maximum']].sample(5))
else:
    print("\n⚠️ No plans scraped")

🧪 TEST: Scraping Albany with the fixed scraper
This should now get all 109 plans!


⚠️ No plans scraped


---
## Full Scrape

**Note:** Scraping all 62 counties will take roughly an hour. Each county has 50-150+ plans.


In [ ]:
# ============================================================================
# BULK SCRAPE: Configure which counties to scrape
# ============================================================================

run_scrape = False
if run_scrape:
    
    # OPTION 1: Scrape ALL 62 NY counties (uncomment the line below)
    counties_to_scrape = NY_COUNTIES

    # OPTION 2: Scrape specific counties
    # counties_to_scrape = ["Albany", "Erie", "Kings", "Queens", "Nassau"]

    # OPTION 3: Scrape first N counties (for testing)
    # counties_to_scrape = NY_COUNTIES[:3]  # First 3 counties

    # OPTION 4: Don't scrape anything (leave empty)
    # counties_to_scrape = []

    coverage_tier = "INDIVIDUAL"
    for plan_year in PLAN_YEARS:

        print(f"📋 Configuration:")
        print(f"   Counties to scrape: {len(counties_to_scrape)}")
        if len(counties_to_scrape) <= 10:
            print(f"   Counties: {', '.join(counties_to_scrape)}")
        print(f"   Coverage tier: {coverage_tier}")
        print(f"   Plan year: {plan_year}")
        print()

        print(f"🚀 Starting bulk scrape...")
        print(f"   Counties: {len(counties_to_scrape)}")
        print(f"   Coverage tier: {coverage_tier}")
        print(f"   Plan year: {plan_year}")
        print(f"\nPress Ctrl+C to stop at any time")
        print(f"{'='*60}\n")

        all_plans_data = []
        failed_counties = []

        for idx, county in enumerate(counties_to_scrape, 1):
            print(f"\n[{idx}/{len(counties_to_scrape)}] County: {county}")
            try:
                plans = scrape_plans_for_county(county, coverage_tier, plan_year)
                if plans:
                    all_plans_data.extend(plans)
                else:
                    failed_counties.append(county)
            except KeyboardInterrupt:
                print("\n\n⚠️ Stopped by user")
                break
            except Exception as e:
                print(f"✗ Error scraping {county}: {e}")
                failed_counties.append(county)
            
            # Progress update
            if idx % 5 == 0:
                print(f"\n{'='*60}")
                print(f"Progress: {idx}/{len(counties_to_scrape)} counties")
                print(f"Total plans collected: {len(all_plans_data)}")
                print(f"{'='*60}\n")

        print(f"\n\n{'='*60}")
        print(f"🎉 SCRAPING COMPLETE!")
        print(f"{'='*60}")
        print(f"Total plans scraped: {len(all_plans_data)}")
        print(f"Counties processed: {len(counties_to_scrape) - len(failed_counties)}/{len(counties_to_scrape)}")
        if failed_counties:
            print(f"Failed counties: {', '.join(failed_counties)}")

        # Convert to DataFrame
        if all_plans_data:
            df_all_plans = pd.DataFrame(all_plans_data)
            print(f"\n✓ Created DataFrame with {len(df_all_plans)} rows × {len(df_all_plans.columns)} columns")
        else:
            df_all_plans = pd.DataFrame()
            print("\n⚠️ No data collected")
        
        # Explore the scraped data
        if not df_all_plans.empty:
            print("📊 DATA SUMMARY")
            print("="*60)
            print(f"Total plans: {len(df_all_plans)}")
            print(f"Columns: {len(df_all_plans.columns)}")
            print(f"\nColumn names:")
            for col in df_all_plans.columns:
                print(f"  - {col}")
            
            print(f"\nPlans by county:")
            county_counts = df_all_plans['county'].value_counts()
            print(county_counts.to_string())
            
            print(f"\n\nFirst 3 rows:")
            display(df_all_plans.head(3))
            
            print(f"\n\nData types:")
            print(df_all_plans.dtypes)
        else:
            print("No data to explore")
        
        # Export data to files
        if not df_all_plans.empty:
            # CSV export
            csv_filename = f'ny_health_plans_{plan_year}_{coverage_tier}.csv'
            df_all_plans.to_csv("output_ny/"+csv_filename, index=False)
            print(f"\n📦 Exported {len(df_all_plans)} plans to csv")
        else:
            print("No data to export")


📋 Configuration:
   Counties to scrape: 62
   Coverage tier: INDIVIDUAL
   Plan year: 2026

🚀 Starting bulk scrape...
   Counties: 62
   Coverage tier: INDIVIDUAL
   Plan year: 2026

Press Ctrl+C to stop at any time

🧵 Using 5 parallel threads


Scraping: Albany | INDIVIDUAL | 2026
  Loading initial page...

Scraping: Allegany | INDIVIDUAL | 2026
  Loading initial page...

Scraping: Bronx | INDIVIDUAL | 2026
  Loading initial page...

Scraping: Broome | INDIVIDUAL | 2026
  Loading initial page...

Scraping: Cattaraugus | INDIVIDUAL | 2026
  Loading initial page...
    ✓ Page 1: 6 plans (Total: 77)    ✓ Page 1: 6 plans (Total: 77)
  Loading page 1...
  Loading page 1...    ✓ Page 1: 6 plans (Total: 102)    ✓ Page 1: 6 plans (Total: 99)
  Loading page 1...
  Loading page 1...    ✓ Page 1: 6 plans (Total: 109)
  Loading page 1... ✓ 6 plans (6 new) ✓ 6 plans (6 new)
 ✓ 6 plans (6 new)
 ✓ 6 plans (6 new)
 ✓ 6 plans (6 new)

  Loading page 2...  Loading page 2...  Loading page 2...  Loading 

,plan_id,full_plan_id,county,coverage_tier,plan_year,plan_name,premium,metal_level,allows_health_savings_account,monthly_premium,persons_covered,individual_annual_deductible,group_annual_deductible,individual_out_of_pocket_maximum,group_out_of_pocket_maximum
0,135116,25303NY0610001,Allegany,INDIVIDUAL,2026,Plan Details,433.76,Bronze,True,433.76,individual,5500.0,5500 per person | 11000 per group,8050.0,8050 per person | 16100 per group
1,135046,25303NY0010001,Allegany,INDIVIDUAL,2026,Plan Details,448.22,Bronze,True,448.22,individual,4125.0,4125 per person | 8250 per group,10150.0,10150 per person | 20300 per group
2,135119,25303NY0620001,Allegany,INDIVIDUAL,2026,Plan Details,455.37,Bronze,True,455.37,individual,5500.0,5500 per person | 11000 per group,8050.0,8050 per person | 16100 per group




Data types:
plan_id                              object
full_plan_id                         object
county                               object
coverage_tier                        object
plan_year                             int64
plan_name                            object
premium                             float64
metal_level                          object
allows_health_savings_account          bool
monthly_premium                     float64
persons_covered                      object
individual_annual_deductible         object
group_annual_deductible              object
individual_out_of_pocket_maximum     object
group_out_of_pocket_maximum          object
dtype: object

📦 Exported 5597 plans to csv
📋 Configuration:
   Counties to scrape: 62
   Coverage tier: INDIVIDUAL
   Plan year: 2025

🚀 Starting bulk scrape...
   Counties: 62
   Coverage tier: INDIVIDUAL
   Plan year: 2025

Press Ctrl+C to stop at any time

🧵 Using 5 parallel threads


Scraping: Albany | INDIVIDUAL | 2025

,plan_id,full_plan_id,county,coverage_tier,plan_year,plan_name,premium,metal_level,allows_health_savings_account,monthly_premium,persons_covered,individual_annual_deductible,group_annual_deductible,individual_out_of_pocket_maximum,group_out_of_pocket_maximum
0,131143,25303NY0610001,Allegany,INDIVIDUAL,2025,Plan Details,429.46,Bronze,True,429.46,individual,5500.0,5500 per person | 11000 per group,8050.0,8050 per person | 16100 per group
1,130988,25303NY0010001,Allegany,INDIVIDUAL,2025,Plan Details,431.66,Bronze,False,431.66,individual,3800.0,3800 per person | 7600 per group,9200.0,9200 per person | 18400 per group
2,131146,25303NY0620001,Allegany,INDIVIDUAL,2025,Plan Details,450.93,Bronze,True,450.93,individual,5500.0,5500 per person | 11000 per group,8050.0,8050 per person | 16100 per group




Data types:
plan_id                              object
full_plan_id                         object
county                               object
coverage_tier                        object
plan_year                             int64
plan_name                            object
premium                             float64
metal_level                          object
allows_health_savings_account          bool
monthly_premium                     float64
persons_covered                      object
individual_annual_deductible         object
group_annual_deductible              object
individual_out_of_pocket_maximum     object
group_out_of_pocket_maximum          object
dtype: object

📦 Exported 5596 plans to csv


In [ ]:
# File reading
import pandas as pd

df = pd.read_csv('output/output_ny/ny_health_plans_2025_INDIVIDUAL.csv')
df.sample(5)

,plan_id,full_plan_id,county,coverage_tier,plan_year,plan_name,premium,metal_level,allows_health_savings_account,monthly_premium,persons_covered,individual_annual_deductible,group_annual_deductible,individual_out_of_pocket_maximum,group_out_of_pocket_maximum
2793,133905,42640NY0360001,Nassau,INDIVIDUAL,2026,Plan Details,40.63,High,False,40.63,individual,See Plan Brochure,See Plan Brochure,See Plan Brochure,See Plan Brochure
3519,134007,56184NY0140015,Otsego,INDIVIDUAL,2026,Plan Details,1238.58,Gold,False,1238.58,individual,775.0,775 per person | 1550 per group,10150.0,10150 per person | 20300 per group
674,137735,78124NY1160006,Chemung,INDIVIDUAL,2026,Plan Details,52.96,High,False,52.96,individual,See Plan Brochure,See Plan Brochure,See Plan Brochure,See Plan Brochure
2369,134016,56184NY0140030,Livingston,INDIVIDUAL,2026,Plan Details,705.19,Bronze,True,705.19,individual,4125.0,4125 per person | 8250 per group,10150.0,10150 per person | 20300 per group
2841,135055,25303NY0030001,New York,INDIVIDUAL,2026,Plan Details,1123.85,Gold,False,1123.85,individual,775.0,775 per person | 1550 per group,10150.0,10150 per person | 20300 per group


In [1]:
# Imports
import pandas as pd
import json
import requests
import random
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time

# State configurations
STATE_CONFIGS = {
    'PA': {
        'name': 'Pennsylvania',
        'base_url': 'https://enroll.pennie.com/prescreener',
        'api_url': 'https://enroll.pennie.com/hix/private/getIndividualPlans'
    },
    'NJ': {
        'name': 'New Jersey',
        'base_url': 'https://enroll.getcovered.nj.gov/prescreener',
        'api_url': 'https://enroll.getcovered.nj.gov/hix/private/getIndividualPlans'
    },
    'VA': {
        'name': 'Virginia',
        'base_url': 'https://enroll.marketplace.virginia.gov/prescreener',
        'api_url': 'https://enroll.marketplace.virginia.gov/hix/private/getIndividualPlans'
    },
    'GA': {
        'name': 'Georgia',
        'base_url': 'https://enroll.georgiaaccess.gov/prescreener',
        'api_url': 'https://enroll.georgiaaccess.gov/hix/private/getIndividualPlans'
    }
}

print(f"✓ Loaded configurations for {len(STATE_CONFIGS)} states")


✓ Loaded configurations for 4 states


In [29]:
# Pennsylvania counties with sample zip codes
# We'll use one representative zip code per county for scraping
PA_COUNTIES_ZIPS = {
    "Adams": "17325",
    "Allegheny": "15213",
    "Armstrong": "16201",
    "Beaver": "15001",
    "Bedford": "15522",
    "Berks": "19601",
    "Blair": "16601",
    "Bradford": "18810",
    "Bucks": "18901",
    "Butler": "16001",
    "Cambria": "15901",
    "Cameron": "15834",
    "Carbon": "18235",
    "Centre": "16801",
    "Chester": "19380",
    "Clarion": "16214",
    "Clearfield": "16830",
    "Clinton": "17745",
    "Columbia": "17815",
    "Crawford": "16335",
    "Cumberland": "17013",
    "Dauphin": "17101",
    "Delaware": "19063",
    "Elk": "15834",
    "Erie": "16501",
    "Fayette": "15401",
    "Forest": "16329",
    "Franklin": "17201",
    "Fulton": "17219",
    "Greene": "15370",
    "Huntingdon": "16652",
    "Indiana": "15701",
    "Jefferson": "15767",
    "Juniata": "17044",
    "Lackawanna": "18503",
    "Lancaster": "17601",
    "Lawrence": "16101",
    "Lebanon": "17042",
    "Lehigh": "18101",
    "Luzerne": "18701",
    "Lycoming": "17701",
    "McKean": "16701",
    "Mercer": "16101",
    "Mifflin": "17044",
    "Monroe": "18301",
    "Montgomery": "19401",
    "Montour": "17815",
    "Northampton": "18042",
    "Northumberland": "17801",
    "Perry": "17018",
    "Philadelphia": "19019",
    "Pike": "18324",
    "Potter": "16901",
    "Schuylkill": "17901",
    "Snyder": "17870",
    "Somerset": "15501",
    "Sullivan": "18614",
    "Susquehanna": "18810",
    "Tioga": "16901",
    "Union": "17837",
    "Venango": "16301",
    "Warren": "16365",
    "Washington": "15301",
    "Wayne": "18431",
    "Westmoreland": "15601",
    "Wyoming": "18610",
    "York": "17401"
}

print(f"✓ Loaded {len(PA_COUNTIES_ZIPS)} Pennsylvania counties with zip codes")

✓ Loaded 67 Pennsylvania counties with zip codes


In [2]:
# New Jersey counties with sample zip codes
# We'll use one representative zip code per county for scraping
NJ_COUNTIES_ZIPS = {
    "Atlantic": "08401",
    "Bergen": "07601",
    "Burlington": "08016",
    "Camden": "08101",
    "Cape May": "08204",
    "Cumberland": "08302",
    "Essex": "07102",
    "Gloucester": "08028",
    "Hudson": "07030",
    "Hunterdon": "08822",
    "Mercer": "08608",
    "Middlesex": "08901",
    "Monmouth": "07701",
    "Morris": "07960",
    "Ocean": "08701",
    "Passaic": "07501",
    "Salem": "08079",
    "Somerset": "08807",
    "Sussex": "07821",
    "Union": "07083",
    "Warren": "07863"
}

print(f"✓ Loaded {len(NJ_COUNTIES_ZIPS)} New Jersey counties with zip codes")

✓ Loaded 21 New Jersey counties with zip codes


In [ ]:
# Georgia counties with sample zip codes
# We'll use one representative zip code per county for scraping
# Source: https://georgiahealthdata.info/Georgia_Zip_Code_County_Lookup.PDF
GA_COUNTIES_ZIPS = {
    "Appling": "31513",
    "Atkinson": "31624",
    "Bacon": "31510",
    "Baker": "39870",
    "Baldwin": "31061",
    "Banks": "30511",
    "Barrow": "30011",
    "Bartow": "30103",
    "Ben Hill": "31750",
    "Berrien": "31622",
    "Bibb": "31201",
    "Bleckley": "31014",
    "Brantley": "31542",
    "Brooks": "31625",
    "Bryan": "31308",
    "Bulloch": "30415",
    "Burke": "30426",
    "Butts": "30216",
    "Calhoun": "39813",
    "Camden": "31547",
    "Candler": "30439",
    "Carroll": "30108",
    "Catoosa": "30736",
    "Charlton": "31537",
    "Chatham": "31302",
    "Chattahoochee": "31805",
    "Chattooga": "30747",
    "Cherokee": "30102",
    "Clarke": "30601",
    "Clay": "39824",
    "Clayton": "30236",
    "Clinch": "31630",
    "Cobb": "30008",
    "Coffee": "31512",
    "Colquitt": "31744",
    "Columbia": "30813",
    "Cook": "31620",
    "Coweta": "30220",
    "Crawford": "31052",
    "Crisp": "31712",
    "Dade": "30731",
    "Dawson": "30534",
    "Decatur": "39815",
    "DeKalb": "30002",
    "Dodge": "31011",
    "Dooly": "31091",
    "Dougherty": "31701",
    "Douglas": "30122",
    "Early": "39823",
    "Echols": "31648",
    "Effingham": "31303",
    "Elbert": "30635",
    "Emanuel": "30401",
    "Evans": "30417",
    "Fannin": "30512",
    "Fayette": "30205",
    "Floyd": "30105",
    "Forsyth": "30040",
    "Franklin": "30562",
    "Fulton": "30004",
    "Gilmer": "30540",
    "Glascock": "30820",
    "Glynn": "31520",
    "Gordon": "30139",
    "Grady": "39827",
    "Greene": "30642",
    "Gwinnett": "30017",
    "Habersham": "30531",
    "Hall": "30501",
    "Hancock": "31087",
    "Haralson": "30110",
    "Harris": "31804",
    "Hart": "30549",
    "Heard": "30217",
    "Henry": "30228",
    "Houston": "31088",
    "Irwin": "31774",
    "Jackson": "30549",
    "Jasper": "31064",
    "Jeff Davis": "31532",
    "Jefferson": "30413",
    "Jenkins": "30442",
    "Johnson": "31096",
    "Jones": "31032",
    "Lamar": "30204",
    "Lanier": "31635",
    "Laurens": "31021",
    "Lee": "31763",
    "Liberty": "31301",
    "Lincoln": "30817",
    "Long": "31316",
    "Lowndes": "31601",
    "Lumpkin": "30533",
    "Macon": "31030",
    "Madison": "30643",
    "Marion": "31803",
    "McDuffie": "30824",
    "McIntosh": "31305",
    "Meriwether": "30218",
    "Miller": "39837",
    "Mitchell": "31716",
    "Monroe": "31029",
    "Montgomery": "30410",
    "Morgan": "30650",
    "Murray": "30705",
    "Muscogee": "31808",
    "Newton": "30014",
    "Oconee": "30677",
    "Oglethorpe": "30648",
    "Paulding": "30132",
    "Peach": "31030",
    "Pickens": "30143",
    "Pierce": "31516",
    "Pike": "30206",
    "Polk": "30104",
    "Pulaski": "31057",
    "Putnam": "31024",
    "Quitman": "39854",
    "Rabun": "30552",
    "Randolph": "39836",
    "Richmond": "30901",
    "Rockdale": "30012",
    "Schley": "31806",
    "Screven": "30467",
    "Seminole": "39845",
    "Spalding": "30223",
    "Stephens": "30577",
    "Stewart": "31815",
    "Sumter": "31709",
    "Talbot": "31801",
    "Taliaferro": "30673",
    "Tattnall": "30420",
    "Taylor": "31006",
    "Telfair": "31544",
    "Terrell": "39826",
    "Thomas": "31626",
    "Tift": "31733",
    "Toombs": "30436",
    "Towns": "30582",
    "Treutlen": "30453",
    "Troup": "30230",
    "Turner": "31714",
    "Twiggs": "31017",
    "Union": "30582",
    "Upson": "30285",
    "Walker": "30741",
    "Walton": "30025",
    "Ware": "31501",
    "Warren": "30828",
    "Washington": "31089",
    "Wayne": "31545",
    "Webster": "31824",
    "Wheeler": "30411",
    "White": "30571",
    "Whitfield": "30720",
    "Wilcox": "31001",
    "Wilkes": "30673",
    "Wilkinson": "31090",
    "Worth": "31772"
}

print(f"✓ Loaded {len(GA_COUNTIES_ZIPS)} Georgia counties with zip codes")

✓ Loaded 159 Georgia counties with zip codes


In [ ]:
# Virginia counties with sample zip codes
# We'll use one representative zip code per county for scraping
VA_COUNTIES_ZIPS = {
    # Counties
    "Accomack": "23301",
    "Albemarle": "22901",
    "Alleghany": "24426",
    "Amelia": "23002",
    "Amherst": "24521",
    "Appomattox": "24522",
    "Arlington": "22201",
    "Augusta": "24401",
    "Bath": "24484",
    "Bedford": "24523",
    "Bland": "24315",
    "Botetourt": "24011",
    "Brunswick": "23821",
    "Buchanan": "24631",
    "Buckingham": "23921",
    "Campbell": "24501",
    "Caroline": "22514",
    "Carroll": "24330",
    "Charles City": "23030",
    "Charlotte": "23947",
    "Chesterfield": "23832",
    "Clarke": "22611",
    "Craig": "24066",
    "Culpeper": "22701",
    "Cumberland": "23040",
    "Dickenson": "24243",
    "Dinwiddie": "23841",
    "Essex": "22560",
    "Fairfax": "22030",
    "Fauquier": "20186",
    "Floyd": "24091",
    "Fluvanna": "22963",
    "Franklin": "24101",
    "Frederick": "22601",
    "Giles": "24126",
    "Gloucester": "23061",
    "Goochland": "23063",
    "Grayson": "24333",
    "Greene": "22968",
    "Greensville": "23875",
    "Halifax": "24558",
    "Hanover": "23069",
    "Henrico": "23228",
    "Henry": "24112",
    "Highland": "24465",
    "Isle of Wight": "23430",
    "James City": "23185",
    "King and Queen": "23085",
    "King George": "22485",
    "King William": "23086",
    "Lancaster": "22503",
    "Lee": "24263",
    "Loudoun": "20147",
    "Louisa": "23093",
    "Lunenburg": "23868",
    "Madison": "22727",
    "Mathews": "23109",
    "Mecklenburg": "23927",
    "Middlesex": "23071",
    "Montgomery": "24060",
    "Nelson": "22949",
    "New Kent": "23124",
    "Northampton": "23389",
    "Northumberland": "22482",
    "Nottoway": "23930",
    "Orange": "22960",
    "Page": "22835",
    "Patrick": "24133",
    "Pittsylvania": "24531",
    "Powhatan": "23139",
    "Prince Edward": "23901",
    "Prince George": "23875",
    "Prince William": "22191",
    "Pulaski": "24301",
    "Rappahannock": "22737",
    "Richmond": "22572",
    "Roanoke": "24019",
    "Rockbridge": "24450",
    "Rockingham": "22801",
    "Russell": "24270",
    "Scott": "24245",
    "Shenandoah": "22842",
    "Smyth": "24377",
    "Southampton": "23837",
    "Spotsylvania": "22553",
    "Stafford": "22554",
    "Surry": "23883",
    "Sussex": "23890",
    "Tazewell": "24651",
    "Warren": "22630",
    "Washington": "24210",
    "Westmoreland": "22443",
    "Wise": "24293",
    "Wythe": "24382",
    "York": "23690",
    
    # Independent Cities
    "Alexandria": "22301",
    "Bristol": "24201",
    "Buena Vista": "24416",
    "Charlottesville": "22901",
    "Chesapeake": "23320",
    "Colonial Heights": "23834",
    "Covington": "24426",
    "Danville": "24541",
    "Emporia": "23847",
    "Fairfax City": "22030",
    "Falls Church": "22046",
    "Franklin City": "23851",
    "Fredericksburg": "22401",
    "Galax": "24333",
    "Hampton": "23661",
    "Harrisonburg": "22801",
    "Hopewell": "23860",
    "Lexington": "24450",
    "Lynchburg": "24501",
    "Manassas": "20110",
    "Manassas Park": "20111",
    "Martinsville": "24112",
    "Newport News": "23601",
    "Norfolk": "23510",
    "Norton": "24273",
    "Petersburg": "23803",
    "Poquoson": "23662",
    "Portsmouth": "23701",
    "Radford": "24141",
    "Richmond City": "23219",
    "Roanoke City": "24011",
    "Salem": "24153",
    "Staunton": "24401",
    "Suffolk": "23434",
    "Virginia Beach": "23451",
    "Waynesboro": "22980",
    "Williamsburg": "23185",
    "Winchester": "22601"
}

✓ Loaded 133 Virginia counties and cities with zip codes


In [3]:
# Generic Functions for Bulk Scraping
def setup_driver(headless=True):
    """Setup Chrome driver with appropriate options"""
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument('--headless=new')
    
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36')
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(10)
    
    return driver

print("✓ Defined setup_driver()")


def navigate_to_plan_finder_generic(driver, state_code, zip_code, county_name, coverage_year, most_recent_year=None):
    """
    Generic navigation function for PA, NJ, VA, and GA prescreeners.
    
    Args:
        driver: Selenium WebDriver instance
        state_code: State code ('PA', 'NJ', 'VA', 'GA')
        zip_code: Zip code to enter
        county_name: County name for selection (if dropdown appears)
        coverage_year: Year to select
        most_recent_year: Most recent available year (if None, won't check for effective date)
    
    Returns:
        bool: True if successful, False otherwise
    """
    
    
    config = STATE_CONFIGS[state_code]
    base_url = config['base_url']
    state_name = config['name']
    
    try:
        print(f"  Navigating to {state_name} prescreener for {county_name} (zip: {zip_code}, year: {coverage_year})...")
        
        # Step 1: Load the prescreener page
        driver.get(base_url)
        time.sleep(2)
        
        # Step 2: Click initial "Continue" button if present
        try:
            initial_continue = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Continue')]"))
            )
            initial_continue.click()
            print(f"    ✓ Clicked initial Continue button")
            time.sleep(1)
        except Exception as e:
            print(f"    ℹ No initial Continue button")
        
        # Step 3: Select coverage year from dropdown
        try:
            year_dropdown = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='coverage-year-dropdown']"))
            )
            select = Select(year_dropdown)
            select.select_by_value(str(coverage_year))
            print(f"    ✓ Selected coverage year: {coverage_year}")
            time.sleep(0.5)
        except Exception as e:
            print(f"    ⚠ Could not select coverage year: {str(e)[:100]}")
        
        # Step 4: If year is not the most recent, select effective date (January by default)
        if most_recent_year and coverage_year < most_recent_year:
            try:
                effective_date_dropdown = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='effective-date-dropdown']"))
                )
                select = Select(effective_date_dropdown)
                # Try to select January (first month)
                try:
                    select.select_by_index(1)  # Usually index 0 is placeholder, 1 is January
                    print(f"    ✓ Selected effective date: January {coverage_year}")
                except:
                    # If that doesn't work, try selecting by visible text
                    for option in select.options:
                        if 'jan' in option.text.lower() or '01' in option.text:
                            select.select_by_visible_text(option.text)
                            print(f"    ✓ Selected effective date: {option.text}")
                            break
                time.sleep(0.5)
            except Exception as e:
                print(f"    ℹ No effective date dropdown found (year is recent or not required)")
        
        # Step 5: Enter zip code
        try:
            zip_selectors = [
                "input[name*='zip' i]",
                "input[id*='zip' i]",
                "input[placeholder*='zip' i]",
                "input[type='text']",
                "input[type='tel']",
            ]
            
            zip_input = None
            for selector in zip_selectors:
                try:
                    zip_input = WebDriverWait(driver, 2).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                    )
                    break
                except:
                    continue
            
            if not zip_input:
                print(f"    ✗ Could not find zip code input")
                return False
            
            zip_input.clear()
            zip_input.send_keys(zip_code)
            print(f"    ✓ Entered zip code: {zip_code}")
            time.sleep(0.5)
            
        except Exception as e:
            print(f"    ✗ Error entering zip code: {str(e)[:100]}")
            return False
        
        # Step 6: Select county from dropdown if it appears
        try:
            county_dropdown = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='zip-county-dropdown'] select"))
            )
            time.sleep(1)
            select = Select(county_dropdown)
            
            # Try to find matching county
            for option in select.options:
                option_text = option.text.strip()
                if county_name.lower() in option_text.lower():
                    select.select_by_visible_text(option_text)
                    print(f"    ✓ Selected county: {option_text}")
                    time.sleep(1)
                    break
            # if county cannot be found, select first option
            if county_name.lower() not in option_text.lower():
                select.select_by_index(0)
                print(f"    ✓ Selected first county: {select.first_selected_option.text}")
                time.sleep(1)
        except:
            print(f"    ℹ No county dropdown found (single county for this zip)")
        
        # Step 7: Enter birthdate
        try:
            birth_year = random.randint(1990, 1995)
            birth_month = random.randint(1, 12)
            birth_day = random.randint(1, 28)
            birthdate = f"{birth_month:02d}/{birth_day:02d}/{birth_year}"
            
            birthdate_selectors = [
                "input[name*='birth' i]",
                "input[id*='birth' i]",
                "input[placeholder*='birth' i]",
                "input[name*='date' i]",
                "input[id*='date' i]",
                "input[type='date']",
            ]
            
            birthdate_input = None
            for selector in birthdate_selectors:
                try:
                    birthdate_input = WebDriverWait(driver, 2).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                    )
                    break
                except:
                    continue
            
            if birthdate_input:
                birthdate_input.clear()
                if birthdate_input.get_attribute('type') == 'date':
                    birthdate_formatted = f"{birth_year}-{birth_month:02d}-{birth_day:02d}"
                    birthdate_input.send_keys(birthdate_formatted)
                else:
                    birthdate_input.send_keys(birthdate)
                print(f"    ✓ Entered birthdate: {birthdate}")
                time.sleep(0.5)
            else:
                print(f"    ⚠ Could not find birthdate input")
            
        except Exception as e:
            print(f"    ⚠ Error entering birthdate: {str(e)[:100]}")
        
        # Step 8: Enter income
        try:
            income_options = [90000, 95000, 100000, 105000, 110000]
            income = random.choice(income_options)
            
            income_selectors = [
                "input[data-testid='household-income-input']",
                "input[id*='income' i]",
                "input[name*='income' i]",
                "input[placeholder*='income' i]",
                "input[type='number']",
            ]
            
            income_input = None
            for selector in income_selectors:
                try:
                    income_input = WebDriverWait(driver, 2).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                    )
                    break
                except:
                    continue
            
            if income_input:
                income_input.clear()
                income_input.send_keys(str(income))
                print(f"    ✓ Entered income: ${income:,}")
                time.sleep(0.5)
            else:
                print(f"    ⚠ Could not find income input")
            
        except Exception as e:
            print(f"    ⚠ Error entering income: {str(e)[:100]}")
        
        # Step 9: Click "Continue" button (after form)
        try:
            continue_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Continue')]"))
            )
            continue_button.click()
            print(f"    ✓ Clicked Continue button (after form)")
            time.sleep(2)
        except Exception as e:
            print(f"    ✗ Error clicking Continue: {str(e)[:100]}")
            return False
        
        # Step 10: Click "Next" button
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Next')]"))
            )
            next_button.click()
            print(f"    ✓ Clicked Next button")
            time.sleep(2)
        except Exception as e:
            print(f"    ⚠ Could not find Next button: {str(e)[:100]}")
        
        # Step 11: Click "Skip to View Plans" button
        try:
            skip_button = None
            
            # Try using the id first
            try:
                skip_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, "skipButton"))
                )
            except:
                # Fallback to XPath selectors
                skip_selectors = [
                    "//button[contains(text(), 'Skip to View Plans')]",
                    "//button[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'skip')]",
                    "//a[contains(text(), 'Skip to View Plans')]",
                ]
                
                for selector in skip_selectors:
                    try:
                        skip_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, selector))
                        )
                        break
                    except:
                        continue
            
            if skip_button:
                skip_button.click()
                print(f"    ✓ Clicked Skip to View Plans button")
                time.sleep(2)
            else:
                print(f"    ⚠ Could not find Skip to View Plans button")
        except Exception as e:
            print(f"    ⚠ Error clicking Skip to View Plans: {str(e)[:100]}")
        
        # Step 12: We should now be on the plan finder page
        print(f"    ✓ Navigation complete. Current URL: {driver.current_url}")
        return True
            
    except Exception as e:
        print(f"  ✗ Failed to navigate: {str(e)[:100]}")
        return False

print("✓ Defined navigate_to_plan_finder_generic()")

def extract_plans_from_api_generic(driver, state_code, county_name, zip_code):
    """
    Generic plan extraction function for PA, NJ, VA, and GA.
    
    Args:
        driver: Selenium WebDriver instance
        state_code: State code ('PA', 'NJ', 'VA', 'GA')
        county_name: County name for data
        zip_code: Zip code for data
    
    Returns:
        list: List of plan dictionaries
    """
    
    
    config = STATE_CONFIGS[state_code]
    api_url = config['api_url']
    
    plans = []
    
    try:
        print(f"    Fetching plans from API...")
        
        # Wait a moment for the page to fully load
        time.sleep(1)
        
        # Get cookies from Selenium driver to use in requests
        selenium_cookies = driver.get_cookies()
        
        # Convert Selenium cookies to requests session cookies
        session = requests.Session()
        for cookie in selenium_cookies:
            session.cookies.set(cookie['name'], cookie['value'])
        
        # Copy headers from the browser
        headers = {
            'User-Agent': driver.execute_script("return navigator.userAgent;"),
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Language': 'en-US,en;q=0.9',
            'X-Requested-With': 'XMLHttpRequest',
            'Referer': driver.current_url,
        }
        
        # Build API URL with timestamp for cache busting
        import time as time_module
        timestamp = int(time_module.time() * 1000)
        full_api_url = f"{api_url}?{timestamp // 1000}.{timestamp % 1000}"
        
        print(f"    API URL: {full_api_url}")
        
        # Make API request
        response = session.get(full_api_url, headers=headers, timeout=30)
        
        if response.status_code != 200:
            print(f"    ✗ API returned status code {response.status_code}")
            return []
        
        # Parse JSON response
        try:
            plans_data = response.json()
            print(f"    ✓ API Response Length: {len(plans_data)} plans")
            
            # Process each plan
            for plan_raw in plans_data:
                try:
                    plan_data = {
                        'county': county_name,
                        'zip_code': zip_code,
                        'planId': plan_raw.get('planId'),
                        'issuerPlanNumber': plan_raw.get('issuerPlanNumber'),
                        'level': plan_raw.get('level'),
                        'deductible': plan_raw.get('deductible'),
                        'oopMax': plan_raw.get('oopMax'),
                        'premiumAfterCredit': plan_raw.get('premiumAfterCredit'),
                        'premiumBeforeCredit': plan_raw.get('premiumBeforeCredit'),
                        'annualPremiumAfterCredit': plan_raw.get('annualPremiumAfterCredit'),
                        'annualPremiumBeforeCredit': plan_raw.get('annualPremiumBeforeCredit'),
                        'qualityRating': plan_raw.get('qualityRating'),
                    }
                    
                    plans.append(plan_data)
                    
                except Exception as e:
                    print(f"      ⚠ Error processing plan: {str(e)[:50]}")
                    continue
            
            print(f"    ✓ Processed {len(plans)} plans")
            return plans
            
        except json.JSONDecodeError as e:
            print(f"    ✗ Failed to parse JSON response: {str(e)[:100]}")
            print(f"    Response text (first 500 chars): {response.text[:500]}")
            return []
        
    except Exception as e:
        print(f"    ✗ Error fetching plans from API: {str(e)[:100]}")
        import traceback
        print(f"    Traceback: {traceback.format_exc()[:200]}")
        return []

print("✓ Defined extract_plans_from_api_generic()")

def scrape_county_generic(state_code, county_name, zip_code, coverage_year, most_recent_year=None, headless=True):
    """
    Generic county scraping function for PA, NJ, VA, and GA.
    
    Args:
        state_code: State code ('PA', 'NJ', 'VA', 'GA')
        county_name: County name
        zip_code: Zip code
        coverage_year: Year to scrape
        most_recent_year: Most recent available year (for effective date dropdown)
        headless: Whether to run browser in headless mode
    
    Returns:
        list: List of plan dictionaries
    """
    config = STATE_CONFIGS[state_code]
    state_name = config['name']
    
    print(f"\n{'='*60}")
    print(f"Scraping: {county_name} | {state_name} | Zip: {zip_code} | Year: {coverage_year}")
    print(f"{'='*60}")
    
    driver = None
    try:
        driver = setup_driver(headless=headless)
        
        # Navigate to plan finder
        success = navigate_to_plan_finder_generic(driver, state_code, zip_code, county_name, coverage_year, most_recent_year)
        
        if not success:
            print(f"  ✗ Failed to navigate to plan finder")
            return []
        
        # Extract plans
        plans = extract_plans_from_api_generic(driver, state_code, county_name, zip_code)
        
        print(f"\n✓ Scraped {len(plans)} plans for {county_name}")
        return plans
        
    except Exception as e:
        print(f"✗ Error scraping {county_name}: {str(e)[:100]}")
        return []
        
    finally:
        if driver:
            driver.quit()

print("✓ Defined scrape_county_generic()")


def get_available_coverage_years_generic(state_code):
    """
    Extract available coverage years for a specific state.
    
    Args:
        state_code: State code ('PA', 'NJ', 'VA', 'GA')
    
    Returns:
        list: Available years as integers, sorted in descending order
    """
    from selenium.webdriver.support.ui import Select
    
    config = STATE_CONFIGS[state_code]
    base_url = config['base_url']
    state_name = config['name']
    
    driver = None
    try:
        print(f"🔍 Extracting available coverage years for {state_name}...")
        print(f"   URL: {base_url}")
        
        driver = setup_driver(headless=True)
        driver.get(base_url)
        time.sleep(2)
        
        # Try to click initial Continue button if present
        try:
            initial_continue = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Continue')]"))
            )
            initial_continue.click()
            print("   ✓ Clicked initial Continue button")
            time.sleep(1)
        except:
            print(f"   ℹ No initial Continue button")
        
        # Find the coverage year dropdown
        try:
            year_dropdown = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='coverage-year-dropdown']"))
            )
            print("   ✓ Found coverage year dropdown")
            
            # Get all options
            select = Select(year_dropdown)
            options = select.options
            
            # Extract years
            years = []
            for option in options:
                year_text = option.text.strip()
                year_value = option.get_attribute('value')
                
                # Try to extract year as integer
                try:
                    if year_text.isdigit():
                        year = int(year_text)
                    elif year_value and year_value.isdigit():
                        year = int(year_value)
                    else:
                        # Try to extract 4-digit year from text
                        import re
                        match = re.search(r'(20\d{2})', year_text)
                        if match:
                            year = int(match.group(1))
                        else:
                            continue
                    
                    if 2020 <= year <= 2030:  # Sanity check
                        years.append(year)
                        print(f"   ✓ Found year: {year}")
                        
                except ValueError:
                    continue
            
            # Sort years in descending order
            years = sorted(set(years), reverse=True)
            
            print(f"\n✅ Successfully extracted {len(years)} available years for {state_name}: {years}")
            return years
            
        except Exception as e:
            print(f"   ✗ Could not find coverage year dropdown: {str(e)[:100]}")
            return []
    
    except Exception as e:
        print(f"✗ Error extracting years: {str(e)[:150]}")
        return []
    
    finally:
        if driver:
            driver.quit()

print("✓ Defined get_available_coverage_years_generic()")


✓ Defined setup_driver()
✓ Defined navigate_to_plan_finder_generic()
✓ Defined extract_plans_from_api_generic()
✓ Defined scrape_county_generic()
✓ Defined get_available_coverage_years_generic()


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from datetime import datetime
import os

def bulk_scrape_state_generic(state_code, counties_dict, max_workers=5, test=False, only_recent_year=False):
    """
    Bulk scrape all counties for a state across all available years using threading.
    Saves a separate CSV file for each year.
    
    Args:
        state_code: State code ('PA', 'NJ', 'VA', 'GA')
        counties_dict: Dictionary of {county_name: zip_code}
        max_workers: Maximum number of parallel threads
        test: Whether to run in test mode
        only_recent_year: Whether to only scrape the most recent year
    
    Returns:
        list: List of saved file paths (one per year)
    """
    import pandas as pd
    
    
    config = STATE_CONFIGS[state_code]
    state_name = config['name']
    
    print(f"\n{'='*80}")
    print(f"🚀 BULK SCRAPING {state_name.upper()}")
    print(f"{'='*80}\n")
    
    # Step 1: Get available coverage years
    available_years = get_available_coverage_years_generic(state_code)
    
    if not available_years:
        print(f"✗ No coverage years found for {state_name}. Aborting.")
        return pd.DataFrame()
    
    most_recent_year = available_years[0]

    # Filter to only most recent year if requested
    if only_recent_year:
        available_years = [most_recent_year]
        print(f"\n📅 Only scraping most recent year: {most_recent_year}")
    else:
        print(f"\n📅 Will scrape {len(available_years)} years: {available_years}")
        print(f"   Most recent year: {most_recent_year}")
    
    # Setup output directory
    output_dir = f"output/output_{state_code.lower()}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Track all saved files
    saved_files = []
    lock = Lock()
    
    # Loop through each year
    for year_idx, coverage_year in enumerate(available_years, 1):
        print(f"\n{'='*80}")
        print(f"📅 SCRAPING YEAR {coverage_year} ({year_idx}/{len(available_years)})")
        print(f"{'='*80}\n")
        
        counties_to_scrape = list(counties_dict.keys())
        print(f"Counties to scrape: {len(counties_to_scrape)}")
        print(f"Max parallel workers: {max_workers}")
        
        # Prepare results for this year
        year_results = []
        completed_count = [0]  # Use list for mutability in nested function
        
        def scrape_county_worker(county_name):
            """Worker function for parallel scraping"""
            zip_code = counties_dict[county_name]
            
            try:
                plans = scrape_county_generic(
                    state_code=state_code,
                    county_name=county_name,
                    zip_code=zip_code,
                    coverage_year=coverage_year,
                    most_recent_year=most_recent_year,
                    headless=True
                )
                
                # Add year to each plan
                for plan in plans:
                    plan['coverage_year'] = coverage_year
                
                with lock:
                    completed_count[0] += 1
                    print(f"\n{'='*80}")
                    print(f"Progress: {completed_count[0]}/{len(counties_to_scrape)} counties completed for year {coverage_year}")
                    print(f"{'='*80}\n")
                
                return plans
                
            except Exception as e:
                print(f"✗ Worker error for {county_name}: {str(e)[:100]}")
                with lock:
                    completed_count[0] += 1
                return []
        
        # Execute parallel scraping for this year
        start_time = datetime.now()
        print(f"\n🏁 Starting parallel scraping for {coverage_year}...\n")
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            future_to_county = {
                executor.submit(scrape_county_worker, county): county 
                for county in counties_to_scrape
            }
            
            # Collect results as they complete
            for future in as_completed(future_to_county):
                county = future_to_county[future]
                try:
                    plans = future.result()
                    year_results.extend(plans)
                except Exception as e:
                    print(f"✗ Exception for {county}: {str(e)[:100]}")
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        # Save data for this year
        print(f"\n{'='*80}")
        print(f"📊 SAVING DATA FOR YEAR {coverage_year}")
        print(f"{'='*80}\n")
        
        if not year_results:
            print(f"⚠️ No results collected for year {coverage_year}!")
        else:
            df_year = pd.DataFrame(year_results)
            
            print(f"✅ Created DataFrame with {len(df_year)} rows for year {coverage_year}")
            print(f"   Columns: {list(df_year.columns)}")
            print(f"   Counties in data: {df_year['county'].nunique()}")
            
            # Save to CSV with year in filename
            if test:
                output_file = f"output/{output_dir}/{state_code.lower()}_health_plans_{coverage_year}_INDIVIDUAL_TEST.csv"
            else:
                output_file = f"output/{output_dir}/{state_code.lower()}_health_plans_{coverage_year}_INDIVIDUAL.csv"
            df_year.to_csv(output_file, index=False)
            saved_files.append(output_file)
            print(f"   💾 Saved to: {output_file}")
        
        print(f"\n{'='*80}")
        print(f"✅ YEAR {coverage_year} COMPLETE")
        print(f"   Time taken: {duration:.1f} seconds ({duration/60:.1f} minutes)")
        print(f"   Plans collected: {len(year_results)}")
        print(f"{'='*80}\n")
    
    # Summary
    print(f"\n{'='*80}")
    print(f"🎉 ALL YEARS COMPLETE FOR {state_name.upper()}")
    print(f"{'='*80}\n")
    print(f"Saved {len(saved_files)} files:")
    for file in saved_files:
        print(f"  - {file}")
    print()
    
    return saved_files

print("✓ Defined bulk_scrape_state_generic()")


✓ Defined bulk_scrape_state_generic()


In [ ]:
# Example: Bulk scrape Pennsylvania (assumes PA_COUNTIES_ZIPS is defined below)

run_pa = False

test = True

if test:
    test_set = random.sample(list(PA_COUNTIES_ZIPS.keys()), 5)
    df_pa = bulk_scrape_state_generic(
        state_code='PA',
        counties_dict={key: PA_COUNTIES_ZIPS[key] for key in test_set},  # Defined in PA section below
        max_workers=5,
        test=test
    )

if run_pa:
    df_pa = bulk_scrape_state_generic(
        state_code='PA',
        counties_dict=PA_COUNTIES_ZIPS,  # Defined in PA section below
        max_workers=5
    )



🚀 BULK SCRAPING PENNSYLVANIA

🔍 Extracting available coverage years for Pennsylvania...
   URL: https://enroll.pennie.com/prescreener
   ✓ Clicked initial Continue button
   ✓ Found coverage year dropdown
   ✓ Found year: 2025
   ✓ Found year: 2024

✅ Successfully extracted 2 available years for Pennsylvania: [2025, 2024]

📅 Will scrape 2 years: [2025, 2024]
   Most recent year: 2025

📅 SCRAPING YEAR 2025 (1/2)

Counties to scrape: 67
Max parallel workers: 5

🏁 Starting parallel scraping for 2025...


Scraping: Adams | Pennsylvania | Zip: 17325 | Year: 2025

Scraping: Allegheny | Pennsylvania | Zip: 15213 | Year: 2025

Scraping: Armstrong | Pennsylvania | Zip: 16201 | Year: 2025

Scraping: Beaver | Pennsylvania | Zip: 15001 | Year: 2025

Scraping: Bedford | Pennsylvania | Zip: 15522 | Year: 2025
  Navigating to Pennsylvania prescreener for Allegheny (zip: 15213, year: 2025)...
  Navigating to Pennsylvania prescreener for Armstrong (zip: 16201, year: 2025)...
  Navigating to Pennsylvan

In [5]:
# Example: Bulk scrape New Jersey (assumes NJ_COUNTIES_ZIPS is defined below)
run_nj = True

test = False

if test:
    test_set = random.sample(list(NJ_COUNTIES_ZIPS.keys()), 5)
    df_nj = bulk_scrape_state_generic(
        state_code='NJ',
        counties_dict={key: NJ_COUNTIES_ZIPS[key] for key in test_set},  # Defined in NJ section below
        max_workers=5,
        test=test
    )

if run_nj:
    df_nj = bulk_scrape_state_generic(
        state_code='NJ',
        counties_dict=NJ_COUNTIES_ZIPS,  # Defined in NJ section below
        max_workers=5
    )



🚀 BULK SCRAPING NEW JERSEY

🔍 Extracting available coverage years for New Jersey...
   URL: https://enroll.getcovered.nj.gov/prescreener
   ℹ No initial Continue button
   ✓ Found coverage year dropdown
   ✓ Found year: 2026
   ✓ Found year: 2025

✅ Successfully extracted 2 available years for New Jersey: [2026, 2025]

📅 Will scrape 2 years: [2026, 2025]
   Most recent year: 2026

📅 SCRAPING YEAR 2026 (1/2)

Counties to scrape: 21
Max parallel workers: 5

🏁 Starting parallel scraping for 2026...


Scraping: Atlantic | New Jersey | Zip: 08401 | Year: 2026

Scraping: Bergen | New Jersey | Zip: 07601 | Year: 2026

Scraping: Burlington | New Jersey | Zip: 08016 | Year: 2026

Scraping: Camden | New Jersey | Zip: 08101 | Year: 2026

Scraping: Cape May | New Jersey | Zip: 08204 | Year: 2026
  Navigating to New Jersey prescreener for Cape May (zip: 08204, year: 2026)...
  Navigating to New Jersey prescreener for Atlantic (zip: 08401, year: 2026)...
  Navigating to New Jersey prescreener for C

In [46]:
# Example: Bulk scrape Virginia (assumes VA_COUNTIES_ZIPS is defined below)
run_va = True

test = False

if test:
    test_set = random.sample(list(VA_COUNTIES_ZIPS.keys()), 5)
    df_va = bulk_scrape_state_generic(
        state_code='VA',
        counties_dict={key: VA_COUNTIES_ZIPS[key] for key in test_set},  # Defined in VA section below
        max_workers=5,
        test=test
    )

if run_va:
    df_va = bulk_scrape_state_generic(
        state_code='VA',
        counties_dict=VA_COUNTIES_ZIPS,  # Defined in VA section below
        max_workers=5
    )



🚀 BULK SCRAPING VIRGINIA

🔍 Extracting available coverage years for Virginia...
   URL: https://enroll.marketplace.virginia.gov/prescreener
   ✓ Clicked initial Continue button
   ✓ Found coverage year dropdown
   ✓ Found year: 2026
   ✓ Found year: 2025

✅ Successfully extracted 2 available years for Virginia: [2026, 2025]

📅 Will scrape 2 years: [2026, 2025]
   Most recent year: 2026

📅 SCRAPING YEAR 2026 (1/2)

Counties to scrape: 133
Max parallel workers: 5

🏁 Starting parallel scraping for 2026...


Scraping: Accomack | Virginia | Zip: 23301 | Year: 2026

Scraping: Albemarle | Virginia | Zip: 22901 | Year: 2026

Scraping: Alleghany | Virginia | Zip: 24426 | Year: 2026

Scraping: Amelia | Virginia | Zip: 23002 | Year: 2026

Scraping: Amherst | Virginia | Zip: 24521 | Year: 2026
  Navigating to Virginia prescreener for Accomack (zip: 23301, year: 2026)...
  Navigating to Virginia prescreener for Amelia (zip: 23002, year: 2026)...
  Navigating to Virginia prescreener for Alleghany (

In [47]:
# Example: Bulk scrape Georgia (assumes GA_COUNTIES_ZIPS is defined below)
run_ga = True

test = False

if test:
    test_set = random.sample(list(GA_COUNTIES_ZIPS.keys()), 5)
    df_ga = bulk_scrape_state_generic(
        state_code='GA',
        counties_dict={key: GA_COUNTIES_ZIPS[key] for key in test_set},  # Defined in GA section below
        max_workers=5,
        test=test
    )

if run_ga:
    df_ga = bulk_scrape_state_generic(
        state_code='GA',
        counties_dict=GA_COUNTIES_ZIPS,  # Defined in GA section below
        max_workers=5
    )



🚀 BULK SCRAPING GEORGIA

🔍 Extracting available coverage years for Georgia...
   URL: https://enroll.georgiaaccess.gov/prescreener
   ℹ No initial Continue button
   ✓ Found coverage year dropdown
   ✓ Found year: 2026
   ✓ Found year: 2025

✅ Successfully extracted 2 available years for Georgia: [2026, 2025]

📅 Will scrape 2 years: [2026, 2025]
   Most recent year: 2026

📅 SCRAPING YEAR 2026 (1/2)

Counties to scrape: 159
Max parallel workers: 5

🏁 Starting parallel scraping for 2026...


Scraping: Appling | Georgia | Zip: 31513 | Year: 2026

Scraping: Atkinson | Georgia | Zip: 31624 | Year: 2026

Scraping: Bacon | Georgia | Zip: 31510 | Year: 2026

Scraping: Baker | Georgia | Zip: 39870 | Year: 2026

Scraping: Baldwin | Georgia | Zip: 31061 | Year: 2026
  Navigating to Georgia prescreener for Baker (zip: 39870, year: 2026)...
  Navigating to Georgia prescreener for Atkinson (zip: 31624, year: 2026)...
  Navigating to Georgia prescreener for Appling (zip: 31513, year: 2026)...
  Navi

---

---
# California Health Plan Scraper


In [48]:
# California Data & Research page URL
ca_base_url = "https://hbex.coveredca.com/data-research/"
print("✓ California base URL defined")


✓ California base URL defined


In [ ]:
def scrape_ca_individual_product_prices():
    """
    Scrape all Excel files with 'Individual Product Prices' in link text
    from Covered California Data & Research page
    """
    import requests
    from bs4 import BeautifulSoup
    import os
    import re
    from urllib.parse import urljoin
    
    print("Fetching Covered California Data & Research page...")
    print(f"URL: {ca_base_url}\n")
    
    try:
        # Fetch the page
        response = requests.get(ca_base_url, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all links with 'Individual Product Prices' in the text
        all_links = soup.find_all('a')
        
        price_files = []
        for link in all_links:
            link_text = link.get_text(strip=True)
            href = link.get('href', '')
            
            # Check if 'Individual Product Prices' is in the link text
            if 'Individual Product Prices' in link_text and href:
                # Make sure it's an Excel file
                if href.endswith(('.xlsx', '.xls')):
                    full_url = urljoin(ca_base_url, href)
                    price_files.append({
                        'text': link_text,
                        'url': full_url,
                        'filename': os.path.basename(href)
                    })
        
        print(f"Found {len(price_files)} Excel files with 'Individual Product Prices'\n")
        print("=" * 80)
        
        # Download each file
        os.makedirs('output/output_ca', exist_ok=True)
        
        downloaded_files = []
        for idx, file_info in enumerate(price_files, 1):
            print(f"\n[{idx}/{len(price_files)}] {file_info['text']}")
            print(f"    URL: {file_info['url']}")
            
            try:
                # Download the file
                file_response = requests.get(file_info['url'], timeout=60)
                file_response.raise_for_status()
                
                # Save to output_ca folder
                output_path = os.path.join('output/output_ca', file_info['filename'])
                with open(output_path, 'wb') as f:
                    f.write(file_response.content)
                
                file_size = len(file_response.content) / 1024  # KB
                print(f"    ✓ Downloaded: {output_path} ({file_size:.1f} KB)")
                downloaded_files.append(output_path)
                
            except Exception as e:
                print(f"    ✗ Error downloading: {str(e)[:100]}")
        
        print("\n" + "=" * 80)
        print(f"✅ Successfully downloaded {len(downloaded_files)}/{len(price_files)} files")
        print(f"Files saved to: output_ca/")
        
        return downloaded_files
        
    except Exception as e:
        print(f"✗ Error scraping page: {str(e)}")
        return []

print("✓ Defined scrape_ca_individual_product_prices()")


✓ Defined scrape_ca_individual_product_prices()


In [50]:
# Run the California scraper
ca_files = scrape_ca_individual_product_prices()


Fetching Covered California Data & Research page...
URL: https://hbex.coveredca.com/data-research/

Found 9 Excel files with 'Individual Product Prices'


[1/9] 2026 QHP Individual Product Prices
    URL: https://www.coveredca.com/pdfs/2026 QHP Individual Product Prices.xlsx
    ✓ Downloaded: output_ca/2026 QHP Individual Product Prices.xlsx (1363.0 KB)

[2/9] 2026 QDP Individual Product Prices
    URL: https://www.coveredca.com/pdfs/2026 QDP Individual Rates_for Press Release_2025.07.30.xlsx
    ✓ Downloaded: output_ca/2026 QDP Individual Rates_for Press Release_2025.07.30.xlsx (81.2 KB)

[3/9] 2025 Individual Product Prices
    URL: https://hbex.coveredca.com/data-research/library/2025 Individual Product Prices.xlsx
    ✓ Downloaded: output_ca/2025 Individual Product Prices.xlsx (1385.0 KB)

[4/9] 2024 Individual Product Prices
    URL: https://hbex.coveredca.com/data-research/library/2024-QHP-Individual-Rates-07-25-2023.xlsx
    ✓ Downloaded: output_ca/2024-QHP-Individual-Rates-07-2